## Import Libraries

In [49]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [50]:
import matplotlib.pyplot as plt
plt.ion()

In [51]:
from pydgilib_extra import *
from atprogram import atprogram

In [52]:
from os import getcwd, path, pardir
import pickle

## Compile and program project

In [53]:
project_path = path.curdir + "/" + "AES-256_Crypto-Accelerator_Countermeasures"
project_name =  "AES-256_Crypto-Accelerator_Countermeasures"
device_name = "ATSAML21J18B"
project_path  

'./AES-256_Crypto-Accelerator_Countermeasures'

In [54]:
atprogram.atprogram(project_path, device_name = device_name ,verbose=3)

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "chiperase"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:57114
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Command "chiperase" finished with return code 0
[DEBUG] Starting execution of "program"
[DEBUG] Memory segment base written at 0x00000000. Size = 0x00001140.
[DEBUG] Memory segment base written at 0x00001140. Size = 0x00000084.
[DEBUG] Command "program" finished with return code 0
[DEBUG] Starting execution of "info"
[ERROR] Error in TCF lockbit format. Missed key Could not get Module LOCKBIT for ATSAML21J18B. (TCF Error code: 131120)
[DEBUG] Command "info" finished with return code 0
[DEBUG] Exit successfully.
Firmware check OK
Chiperase completed successfully
Programming completed successfully.
Tool edbg has firmware version: 03.25
Target voltage: 3.31 V

Device information:

Name:       ATSAML21J18B 

0

In [55]:
#atprogram(path.abspath(path.join(*project_path)), verbose=2)

In [56]:
atprogram.get_project_size(project_path, device_name = device_name, verbose=2)

{'text': 4416, 'data': 132, 'bss': 8332, 'dec': 12880, 'hex': 12880, 'filename': 'AES-256_Crypto-Accelerator_Countermeasures.elf'}


{'text': 4416,
 'data': 132,
 'bss': 8332,
 'dec': 12880,
 'hex': 12880,
 'filename': 'AES-256_Crypto-Accelerator_Countermeasures.elf'}

## Data Logging

In [57]:
live_plot = False

Create a figure for the plot.

In [58]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [59]:
config_dict = {
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "file_name_base": "experiment_aes_flash"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 5,
    "window_title": "Experiment AES-128 Flash",
}

Stop criteria to pass to the logger:

In [60]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

In [61]:
#with DGILib() as dgilib:
#     dgilib.get_major_version()

Perform the measurement.

In [62]:
data = []
cd = config_dict.copy()
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
    
    cd.update(config_dict_plot)
    cd["fig"] = fig
    
with DGILibExtra(**cd) as dgilib:
    dgilib.device_reset()
    dgilib.logger.log(1000,stop_fn)
    data = dgilib.data

In [63]:
data.length()

{48: 6005, 256: 900000}

In [64]:
print(data)

Interfaces:
	  48:   gpio,    samples:    6005
	 256:   power,   samples:  900000



# Store Data

In [97]:
import pickle

In [98]:
pickle.dump(data, open("aes_flash_logger_data.p", "wb"))

# Load Data

In [99]:
data = pickle.load(open("aes_flash_logger_data.p", "rb"))

In [100]:
#iteration = 0
#name = "AES-128_Flash"
#data = pickle.load(open(path.join(path.pardir, path.pardir, f"{name}_{iteration}.p"), "rb"))

## Analysis

Create Stop Function to stop parsing the data when all pins are high.

In [101]:
def stop_function(pin_values):
    return all(pin_values)

Parse the data.

In [102]:
aes_charge, aes_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)

In [103]:
flash_charge, flash_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

In [104]:
print(len(aes_charge), len(aes_time), len(flash_charge), len(flash_time))
# cutoff = min(len(aes_charge), len(aes_time), len(flash_charge), len(flash_time))
# aes_charge = aes_charge[:cutoff]
# aes_time = aes_time[:cutoff]
# flash_charge = flash_charge[:cutoff]
# flash_time = flash_time[:cutoff]

750 750 750 750


In [105]:
# length = len(aes_charge)
# assert length == len(aes_time)
# assert length == len(flash_charge)
# assert length == len(flash_time)
# print(length)

In [106]:
aes_encrypt_charge = aes_charge[0::2]
aes_decrypt_charge = aes_charge[1::2]
aes_encrypt_time = aes_time[0::2]
aes_decrypt_time = aes_time[1::2]

In [107]:
aes_flash_write_charge = flash_charge[0::2]
aes_flash_read_charge = flash_charge[1::2]
aes_flash_write_time = flash_time[0::2]
aes_flash_read_time = flash_time[1::2]

In [108]:
len(aes_encrypt_charge), len(aes_decrypt_charge), len(aes_encrypt_time), len(aes_decrypt_time), len(aes_flash_write_charge), len(aes_flash_read_charge), len(aes_flash_write_time), len(aes_flash_read_time)

(375, 375, 375, 375, 375, 375, 375, 375)

In [109]:
drop = 0
cutoff = min(len(aes_encrypt_charge), len(aes_decrypt_charge), len(aes_encrypt_time), len(aes_decrypt_time), len(aes_flash_write_charge), len(aes_flash_read_charge), len(aes_flash_write_time), len(aes_flash_read_time)) - drop
aes_encrypt_charge = aes_encrypt_charge[:cutoff]
aes_decrypt_charge = aes_decrypt_charge[:cutoff]
aes_encrypt_time = aes_encrypt_time[:cutoff]
aes_decrypt_time = aes_decrypt_time[:cutoff]
aes_flash_write_charge = aes_flash_write_charge[:cutoff]
aes_flash_read_charge = aes_flash_read_charge[:cutoff]
aes_flash_write_time = aes_flash_write_time[:cutoff]
aes_flash_read_time = aes_flash_read_time[:cutoff]

In [110]:
length = len(aes_encrypt_charge)
assert length == len(aes_decrypt_charge)
assert length == len(aes_encrypt_time)
assert length == len(aes_decrypt_time)
assert length == len(aes_flash_write_charge)
assert length == len(aes_flash_read_charge)
assert length == len(aes_flash_write_time)
assert length == len(aes_flash_read_time)
print(length)

375


# Convert to Joule

In [111]:
voltage = 3.31
j_scale = 1e3 # m
t_scale = 1e3 # m
model_j_scale = 1e6 # n
model_t_scale = 1e3 # u

experiment_name = project_name

In [112]:
aes_encrypt_energy = aes_encrypt_charge[:cutoff]
aes_flash_write_energy = aes_flash_write_charge[:cutoff]
aes_flash_read_energy = aes_flash_read_charge[:cutoff]
aes_decrypt_energy = aes_decrypt_charge[:cutoff]
aes_encrypt_time_s = aes_encrypt_time[:cutoff]
aes_flash_write_time_s = aes_flash_write_time[:cutoff]
aes_flash_read_time_s = aes_flash_read_time[:cutoff]
aes_decrypt_time_s = aes_decrypt_time[:cutoff]

for i in range(len(aes_encrypt_energy)):
    aes_encrypt_energy[i] = aes_encrypt_energy[i] * voltage * j_scale
for i in range(len(aes_flash_write_energy)):
    aes_flash_write_energy[i] = aes_flash_write_energy[i] * voltage * j_scale
for i in range(len(aes_flash_read_energy)):
    aes_flash_read_energy[i] = aes_flash_read_energy[i] * voltage * j_scale
for i in range(len(aes_decrypt_energy)):
    aes_decrypt_energy[i] = aes_decrypt_energy[i] * voltage * j_scale
    
for i in range(len(aes_encrypt_time_s)):
    aes_encrypt_time_s[i] = aes_encrypt_time_s[i] * t_scale
for i in range(len(aes_flash_write_time_s)):
    aes_flash_write_time_s[i] = aes_flash_write_time_s[i] * t_scale
for i in range(len(aes_flash_read_time_s)):
    aes_flash_read_time_s[i] = aes_flash_read_time_s[i] * t_scale
for i in range(len(aes_decrypt_time_s)):
    aes_decrypt_time_s[i] = aes_decrypt_time_s[i] * t_scale
aes_encrypt_energy[255]     

0.050390766195374595

In [81]:
MBEDTLS_AES_BLOCK_SIZE = 16
STEP_SIZE = MBEDTLS_AES_BLOCK_SIZE
MIN_NUM_BYTES = STEP_SIZE
num_bytes = range(MIN_NUM_BYTES, MIN_NUM_BYTES + STEP_SIZE * len(aes_encrypt_energy), STEP_SIZE)
print(f"MAX_NUM_BYTES: {num_bytes[-1]}")

MAX_NUM_BYTES: 6000


In [82]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)
# pars['intercept'].set(min=0)

In [83]:
results = []
ylabels = (['Energy [mJ]'] * 2 + ['Time [ms]'] * 2) * 2 + ['Energy [mJ]'] + ['Time [ms]']
parameter_names = [
    'Encrypt Energy',
    'Flash Write Energy',
    'Flash Read Energy',
    'Decrypt Energy',
    'Encrypt Time',
    'Flash Write Time',
    'Flash Read Time',
    'Decrypt Time',
    'Total Energy',
    'Total Time',
]
for y in [aes_encrypt_energy, aes_flash_write_energy, aes_flash_read_energy, aes_decrypt_energy, aes_encrypt_time_s, aes_flash_write_time_s, aes_flash_read_time_s, aes_decrypt_time_s,
          [e + w + r + d for (e,w,r,d) in zip(aes_encrypt_energy, aes_flash_write_energy, aes_flash_read_energy, aes_decrypt_energy)],
          [e + w + r + d for (e,w,r,d) in zip(aes_encrypt_time_s, aes_flash_write_time_s, aes_flash_read_time_s, aes_decrypt_time_s)]]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    fig, grid = result.plot(
        xlabel='Checkpoint Size [Bytes]',
        ylabel=ylabels[len(results)])
    fig.tight_layout(rect=(0.05, 0.05, 1, 1))
    fig.set_size_inches(5, 4.5, forward=True)
    fig.canvas.set_window_title(
        f"Residuals of {experiment_name} {parameter_names[len(results)]}")
    fig.savefig( f"Residuals of {experiment_name} {parameter_names[len(results)]}")
    fig.show()
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 375
    # variables        = 2
    chi-square         = 3.7198e-06
    reduced chi-square = 9.9728e-09
    Akaike info crit   = -6906.78290
    Bayesian info crit = -6898.92905
[[Variables]]
    slope:      1.2248e-05 +/- 2.9774e-09 (0.02%) (init = 0)
    intercept:  3.0854e-04 +/- 1.0335e-05 (3.35%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.867
[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 375
    # variables        = 2
    chi-square         = 0.02312636
    reduced chi-square = 6.2001e-05
    Akaike info crit   = -3631.14051
    Bayesian info crit = -3623.28666
[[Variables]]
    slope:      1.1735e-04 +/- 2.3476e-07 (0.20%) (init = 0)
    intercept:  0.01574493 +/- 8.1486e-04 (5.18%) (init = 1)
[[Correlations]] (unreport

In [84]:
fig2 = plt.figure(figsize=(8, 6))
fig2.canvas.set_window_title(f"Analysis {experiment_name}")

In [85]:
charge_color = 'r'
time_color = 'b'

In [86]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Checkpoint Size [Bytes]')
ax1.set_ylabel('Energy [mJ]', color=charge_color)
ax2.set_ylabel('Time [ms]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [87]:
lines = []
lines += ax1.plot(num_bytes, aes_encrypt_energy, charge_color+'-', label=f'{parameter_names[len(lines)]}')
lines += ax1.plot(num_bytes, aes_flash_write_energy, charge_color+'-.', label=f'{parameter_names[len(lines)]}')
lines += ax1.plot(num_bytes, aes_flash_read_energy, charge_color+':', label=f'{parameter_names[len(lines)]}')
lines += ax1.plot(num_bytes, aes_decrypt_energy, charge_color+'--', label=f'{parameter_names[len(lines)]}')
lines += ax2.plot(num_bytes, aes_encrypt_time_s, time_color+'-', label=f'{parameter_names[len(lines)]}')
lines += ax2.plot(num_bytes, aes_flash_write_time_s, time_color+'-.', label=f'{parameter_names[len(lines)]}')
lines += ax2.plot(num_bytes, aes_flash_read_time_s, time_color+':', label=f'{parameter_names[len(lines)]}')
lines += ax2.plot(num_bytes, aes_decrypt_time_s, time_color+'--', label=f'{parameter_names[len(lines)]}')
ax1.legend(handles=lines)
ax1.set_title(
    f"{parameter_names[0]}: Slope {results[0].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[0].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[1]}: Slope {results[1].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[1].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[2]}: Slope {results[2].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[2].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[3]}: Slope {results[3].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[3].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[4]}: Slope {results[4].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[4].params['intercept'].value * model_t_scale:.04} $\mu$s\n" +
    f"{parameter_names[5]}: Slope {results[5].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[5].params['intercept'].value * model_t_scale:.04} $\mu$s\n" +
    f"{parameter_names[6]}: Slope {results[6].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[6].params['intercept'].value * model_t_scale:.04} $\mu$s\n" +
    f"{parameter_names[7]}: Slope {results[7].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[7].params['intercept'].value * model_t_scale:.04} $\mu$s\n" +
    f"{parameter_names[8]}: Slope {results[8].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[8].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[9]}: Slope {results[9].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[9].params['intercept'].value * model_t_scale:.04} $\mu$s\n")
fig2.tight_layout()
fig2.savefig(f"Analysis {experiment_name}")
fig2.show()

In [88]:
print(
    f"{parameter_names[0]}: Slope {results[0].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[0].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[1]}: Slope {results[1].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[1].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[2]}: Slope {results[2].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[2].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[3]}: Slope {results[3].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[3].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[4]}: Slope {results[4].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[4].params['intercept'].value * model_t_scale:.020} $\mu$s\n" +
    f"{parameter_names[5]}: Slope {results[5].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[5].params['intercept'].value * model_t_scale:.020} $\mu$s\n" +
    f"{parameter_names[6]}: Slope {results[6].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[6].params['intercept'].value * model_t_scale:.020} $\mu$s\n" +
    f"{parameter_names[7]}: Slope {results[7].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[7].params['intercept'].value * model_t_scale:.020} $\mu$s\n" +
    f"{parameter_names[8]}: Slope {results[8].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[8].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[9]}: Slope {results[9].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[9].params['intercept'].value * model_t_scale:.020} $\mu$s\n"
)

Encrypt Energy: Slope 12.24825924335437044 nJ/B, Intercept 308.5356802475791369 nJ
Flash Write Energy: Slope 117.34540639207232005 nJ/B, Intercept 15744.930328546070086 nJ
Flash Read Energy: Slope 3.3210634345836180792 nJ/B, Intercept -52.419191112429430746 nJ
Decrypt Energy: Slope 12.202011610463063818 nJ/B, Intercept 430.74661377284775199 nJ
Encrypt Time: Slope 9.3178833818385147225 $\mu$s/B, Intercept 204.59929522244991063 $\mu$s
Flash Write Time: Slope 21.781188148740977084 $\mu$s/B, Intercept 2619.8932700669088263 $\mu$s
Flash Read Time: Slope 2.1251993380711713399 $\mu$s/B, Intercept 21.575098293758255608 $\mu$s
Decrypt Time: Slope 9.2882932466769876356 $\mu$s/B, Intercept 282.11510089939650925 $\mu$s
Total Energy: Slope 145.11674068047332753 nJ/B, Intercept 16431.793437997497676 nJ
Total Time: Slope 42.5125640773794089 $\mu$s/B, Intercept 3128.182974791180186 $\mu$s



In [89]:
# Save Charge amount list into pickle file
import pickle
pickle.dump(aes_encrypt_energy, open("aes_flash_encrypt_energy_mJ.p", "wb"))
pickle.dump(aes_decrypt_energy, open("aes_flash_decrypt_energy_mJ.p", "wb"))
pickle.dump(aes_flash_write_energy, open("aes_flash_write_energy_mJ.p", "wb"))
pickle.dump(aes_flash_read_energy, open("aes_flash_read_energy_mJ.p", "wb"))
pickle.dump(aes_encrypt_time_s, open("aes_flash_encrypt_time_ms.p", "wb"))
pickle.dump(aes_decrypt_time_s, open("aes_flash_decrypt_time_ms.p", "wb"))
pickle.dump(aes_flash_write_time_s, open("aes_flash_write_time_ms.p", "wb"))
pickle.dump(aes_flash_read_time_s, open("aes_flash_read_time_ms.p", "wb"))

In [90]:
aes = [aes_encrypt_energy, aes_flash_write_energy, aes_flash_read_energy, aes_decrypt_energy, aes_encrypt_time_s, aes_flash_write_time_s, aes_flash_read_time_s, aes_decrypt_time_s]
for i in aes:
    print(len(i), len(i)*16)

375 6000
375 6000
375 6000
375 6000
375 6000
375 6000
375 6000
375 6000


## Write config file

In [91]:
import json

config = {}
config["name"] = "AES-256 Flash"
config["project_paths"] = [project_path]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["AES-256 Encrypt", "AES-256 Decrypt"], 3: ["AES-256 Flash Write", "AES-256 Flash Read"]}, 
                      "result_types": ["Charge", "Time"],
                      "section_types": {"init": [], 
                                        "store": ["AES-256 Encrypt", "AES-256 Flash Write"],
                                        "load": ["AES-256 Flash Read", "AES-256 Decrypt"],
                                        "exit": []},
                      "labels": {
                          "Charge": {"x":"Data Size", "x_unit": "byte", "y": "Charge", "y_unit": "C"},
                          "Time": {"x":"Data Size", "x_unit": "byte", "y": "Time", "y_unit": "s"},
                      },
                      "x_step": MBEDTLS_AES_BLOCK_SIZE}

with open("looped_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)

# Write model data

In [92]:
dump_pickle = True
fit_lm = True
verbose = 2
show_lm_plot = 2
# drop = 1

In [113]:
# Parse data
analysis_config = config.get("analysis")
result_types = analysis_config.get("result_types")
x_step = analysis_config.get("x_step")
parsed_data = {}
for pin, parameter_names in analysis_config.get("pins").items():
    data2 = power_and_time_per_pulse(
        data, int(pin), stop_function=stop_function)
    num_names = len(parameter_names)
    for i, parameter_name in enumerate(parameter_names):
        end_index = -drop * num_names or None
        parsed_data[parameter_name] = {
            result_types[0]: data2[0][i:end_index:num_names],
            result_types[1]: data2[1][i:end_index:num_names],
            "x_step": x_step}
if dump_pickle:
    pickle.dump(parsed_data, open(
        path.join(path.curdir,
                  f"{config_dict.get('file_name_base')}_looped.p"), "wb"))
#parsed_data    

{'AES-256 Encrypt': {'Charge': [1.3903857387683034e-07,
   2.3141990930642955e-07,
   2.575479606085443e-07,
   3.667259000350069e-07,
   3.965865344853688e-07,
   4.469185631719987e-07,
   5.243595662339958e-07,
   5.775419414373728e-07,
   6.540657766145543e-07,
   6.811241344564381e-07,
   7.614399835205023e-07,
   8.127625299556945e-07,
   8.622188085412305e-07,
   9.358534621099167e-07,
   9.909036622608477e-07,
   1.047904647121402e-06,
   1.1216218927664757e-06,
   1.149513597241685e-06,
   1.2017642145123415e-06,
   1.3119803475896117e-06,
   1.3325092010503626e-06,
   1.389317846245423e-06,
   1.4620960784835018e-06,
   1.5152801670619216e-06,
   1.5881995703724622e-06,
   1.6441878017993498e-06,
   1.7028103169333094e-06,
   1.744966626820099e-06,
   1.7990885807925964e-06,
   1.8819552978414735e-06,
   1.9304737278898023e-06,
   1.983849754874753e-06,
   2.069480966986685e-06,
   2.0956060683414503e-06,
   2.1609362787829134e-06,
   2.2479094212986455e-06,
   2.3018263305783

In [94]:
# Fit lm
if fit_lm:
    model = None
    if model is None:
        def line(x, intercept, slope):
            """a line"""
            return [intercept + slope*i for i in x]

        model = Model(line)
        params = model.make_params(intercept=0, slope=1)
#         params['intercept'].set(min=0)
    else:
        params = model.params

    model_results = {}
    labels = analysis_config.get("labels")
    for parameter_name in parsed_data.keys():
        length = len(parsed_data[parameter_name][result_types[0]])
        x_step = parsed_data[parameter_name]["x_step"]
        num_bytes = range(x_step, (length+1)*x_step, x_step)
        if verbose:
            print(
                f"Fitting model to {parameter_name} with {length} " +
                f"samples, from {min(num_bytes)} to {max(num_bytes)} "
                f"bytes in steps of {x_step}.")
        model_result = {}
        for result_type in result_types:
            model_result[result_type] = model.fit(
                parsed_data[parameter_name][result_type], params,
                x=num_bytes)
            if verbose >= 2:
                print(model_result[result_type].fit_report())
            # Plot multiple view
            if show_lm_plot >= 2:
                fig, grid = model_result[result_type].plot(
                    xlabel=f"{labels[result_type]['x']} " +
                           f"[{labels[result_type]['x_unit']}]",
                    ylabel=f"{labels[result_type]['y']} " +
                           f"[{labels[result_type]['y_unit']}]")
                fig.canvas.set_window_title(
                    f"Residuals of {parameter_name}")
                fig.tight_layout()
                fig.show()
            model_results[parameter_name] = model_result

    # Plot single view
    if show_lm_plot:
        import matplotlib.pyplot as plt
        fig = plt.figure(figsize=(9, 6))
        fig.canvas.set_window_title(f"Analysis {config.get('name')}")
        colors = dict(zip(result_types, ['r', 'b']))
        line_styles = (
            line_style for line_style in ('-', '--', '-.', ':') * 2)
        # fig.suptitle(f"Energy analysis of {config.get('name')}")
        ax = {}
        ax[result_types[0]] = fig.add_subplot(1, 1, 1)
        ax[result_types[1]] = ax[result_types[0]].twinx()
        ax[result_types[0]].set_xlabel(
            f"{labels[result_types[0]]['x']} " +
            f"[{labels[result_types[0]]['x_unit']}]")
        for result_type in result_types:
            ax[result_type].set_ylabel(
                f"{labels[result_type]['y']} " +
                f"[{labels[result_type]['y_unit']}]",
                color=colors[result_type])
            ax[result_type].tick_params('y', colors=colors[result_type])

        lines = []
        title_str = ""
        for parameter_name in parsed_data.keys():
            length = len(parsed_data[parameter_name][result_types[0]])
            x_step = parsed_data[parameter_name]["x_step"]
            num_bytes = range(x_step, (length+1)*x_step, x_step)
            model_result = {}
            line_style = next(line_styles)
            for result_type in result_types:
                label = f"{parameter_name} {labels[result_type]['y']}"
                lines += ax[result_type].plot(
                    num_bytes, parsed_data[parameter_name][result_type],
                    colors[result_type] + line_style, label=label)
                title_str += f"{label} "
                for param in params.keys():
                    title_str += "".join(
                        f"{params[param].name.capitalize()}: ")
                    title_str += "".join(
                        f"{model_results[parameter_name][result_type].params[param].value: .03} ")
                    title_str += "".join(
                        f"{labels[result_type]['y_unit']}, ")
                title_str = title_str[:-2] + \
                    f" per {labels[result_type]['x_unit']}\n"
        ax[result_types[0]].legend(handles=lines)
        ax[result_types[0]].set_title(title_str[:-1])
        # fig.tight_layout()
        fig.tight_layout(rect=(0.05, 0.05, 1, 1))
        fig.set_size_inches(8, 6, forward=True)
        fig.show()

    # Save model results to file
    if dump_pickle:
        model_results_dump = {}
        for parameter_name in model_results.keys():
            model_results_dump[parameter_name] = {}
            for result_type in model_results[parameter_name].keys():
                model_results_dump[parameter_name][result_type] = \
                    model_results[parameter_name][result_type].values

        pickle.dump(model_results_dump, open(path.join(
            path.curdir,
            f"{config_dict.get('file_name_base')}_model.p"), "wb"))

Fitting model to AES-256 Encrypt with 375 samples, from 16 to 6000 bytes in steps of 16.
[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 10
    # data points      = 375
    # variables        = 2
    chi-square         = 3.3952e-13
    reduced chi-square = 9.1025e-16
    Akaike info crit   = -12985.3105
    Bayesian info crit = -12977.4566
[[Variables]]
    intercept:  9.3213e-08 +/- 3.1222e-09 (3.35%) (init = 0)
    slope:      3.7004e-09 +/- 8.9951e-13 (0.02%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(intercept, slope) = -0.867
[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 375
    # variables        = 2
    chi-square         = 1.9129e-06
    reduced chi-square = 5.1283e-09
    Akaike info crit   = -7156.19062
    Bayesian info crit = -7148.33676
[[Variables]]
    intercept:  2.0460e-04 +/- 7.4109e-06 (3.62%) (init = 

# Total from measurement

In [95]:
n_samples = 5995

total_energy = sum(aes_encrypt_energy[:n_samples]) + sum(aes_flash_write_energy[:n_samples]) + sum(aes_flash_read_energy[:n_samples]) + sum(aes_decrypt_energy[:n_samples])
total_time = sum(aes_encrypt_time_s[:n_samples]) + sum(aes_flash_write_time_s[:n_samples]) + sum(aes_flash_read_time_s[:n_samples]) + sum(aes_decrypt_time_s[:n_samples])
print(total_energy, total_time)

169.85360602762094 49127.24085866666


In [96]:
          [e + w + r + d for (e,w,r,d) in zip(aes_encrypt_energy, aes_flash_write_energy, aes_flash_read_energy, aes_decrypt_energy)],
          [e + w + r + d for (e,w,r,d) in zip(aes_encrypt_time_s, aes_flash_write_time_s, aes_flash_read_time_s, aes_decrypt_time_s)]

[5.148332800000002,
 5.610362666666668,
 5.940383999999993,
 6.138396800000001,
 6.798439466666695,
 7.1935349333333,
 7.325526400000032,
 7.8535338666666785,
 8.579653333333326,
 8.579653333333326,
 9.30652639999996,
 9.702548799999988,
 9.965865599999901,
 10.295667199999968,
 10.955885866666682,
 11.352550399999993,
 16.236493866666592,
 16.499333333333254,
 16.830006400000098,
 17.35884160000012,
 17.819690666666776,
 18.347406933333254,
 18.80939200000009,
 19.139792000000046,
 19.33909386666677,
 19.99810720000006,
 20.459173333333357,
 20.857179733333453,
 21.31866826666673,
 21.515130666666547,
 22.308794666666905,
 22.30876959999994,
 27.32267519999998,
 27.322853866666552,
 28.181459733332968,
 28.249369599999685,
 28.774837333333636,
 29.304640000000106,
 29.4368362666666,
 30.02830133333311,
 30.427721066667246,
 30.623575466666832,
 31.1508069333335,
 31.679421866666722,
 31.879803200000055,
 32.471249599999915,
 32.802598400000214,
 33.2647424000001,
 37.949093333333295,
